In [1]:
# 搗包
import os
import requests
from bs4 import BeautifulSoup
import urllib.parse
import time
import opencc

# 挑選字符

In [18]:
字頻 = []
with open('characters_traditional_frequencies.csv', 'r',encoding='utf8') as f: # 您瞧這名字多長
    lines = f.readlines()
    print(len(lines))
    for line in lines[0:6000]:
        字頻.append(line.split(',')[0].strip())
print(len(字頻))
字頻[0:5]
# 第一個是什麽鬼
字頻.pop(0)
字頻[0:5]

13061
6000


['的', '是', '不', '我', '一']

# 爬取字符

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'}

In [5]:
# 字頻[799]
# # 確認某個位置是不是某個字
# # double-check

In [23]:
字頻.index('璫')
# # 這個用來檢查現在爬蟲到第幾個字了
# # 在爬蟲過程中我是在另外一個ipynb裏檢查的

5998

In [22]:
# for zi in 字頻[79:100]:
for zi in 字頻[5700:6000]:
#     print(zi)
    zi_parse = urllib.parse.quote(zi)
    url = 'https://www.zdic.net/zd/zx/jw/' + zi_parse
    r = requests.get(url,headers=headers)
    text = r.text
    soup = BeautifulSoup(text, 'html.parser')
    try:
        jw = soup.find(name='div',attrs={"class": "zy1"})
        img_list = jw.find_all(name='img')
        
        headers['Referer'] = url

        for img in img_list[0:1]:
            # 暫時只要第一個
            src = 'https:'+ img.get('data-original')
            res = requests.get(url=src,headers=headers)
            filename = zi+'.svg'
            with open(filename,'wb') as f:
                f.write(res.content)        
    except:
        print(zi+'沒有金文')
        time.sleep(8)
        continue
    time.sleep(9)

苜沒有金文
桅沒有金文
砭沒有金文
訌沒有金文
笞沒有金文
鈽沒有金文
奭沒有金文
縊沒有金文
壙沒有金文
奼沒有金文
柸沒有金文
枵沒有金文
洘沒有金文
圇沒有金文
祤沒有金文
菘沒有金文
嫋沒有金文
煲沒有金文
煐沒有金文
睒沒有金文
蒔沒有金文
嘳沒有金文
薙沒有金文
鍠沒有金文
鶡沒有金文
仳沒有金文
牝沒有金文
氤沒有金文
匏沒有金文
堰沒有金文
閎沒有金文
煬沒有金文
嬈沒有金文
縉沒有金文
鷂沒有金文
蘸沒有金文
扙沒有金文
妗沒有金文
沏沒有金文
玓沒有金文
苺沒有金文
栴沒有金文
烔沒有金文
蚋沒有金文
凐沒有金文
詎沒有金文
嫈沒有金文
廕沒有金文
漼沒有金文
璆沒有金文
澪沒有金文
顗沒有金文
鶇沒有金文
鶖沒有金文
灩沒有金文
怩沒有金文
剜沒有金文
戛沒有金文
橈沒有金文
优沒有金文
冱沒有金文
呲沒有金文
怹沒有金文
苾沒有金文
苡沒有金文
砢沒有金文
茩沒有金文
痟沒有金文
誒沒有金文
諢沒有金文
擣沒有金文
闓沒有金文
韙沒有金文
蠓沒有金文
臠沒有金文
胰沒有金文
篙沒有金文
闈沒有金文
櫺沒有金文
蘚沒有金文
芼沒有金文
倢沒有金文
砵沒有金文
悾沒有金文
捵沒有金文
珽沒有金文
蚼沒有金文
傒沒有金文
棑沒有金文
硞沒有金文
寘沒有金文
楂沒有金文
稙沒有金文
暪沒有金文
謖沒有金文
嚙沒有金文
櫫沒有金文
蘶沒有金文
汛沒有金文
峒沒有金文
漯沒有金文
擭沒有金文
聱沒有金文
黷沒有金文
俇沒有金文
姽沒有金文
萁沒有金文
亄沒有金文
葥沒有金文
潳沒有金文
魠沒有金文
獞沒有金文
磲沒有金文
鴽沒有金文
鐿沒有金文
鶼沒有金文
枓沒有金文
琺沒有金文
稈沒有金文
櫝沒有金文
襖沒有金文
鐫沒有金文
吨沒有金文
柀沒有金文
枳沒有金文
秏沒有金文
淯沒有金文
婐沒有金文
掮沒有金文
莪沒有金文
堸沒有金文
渼沒有金文
篫沒有金文
諤沒有金文
艟沒有金文
龕沒有金文
佾沒有金文
隄沒有金文
楛沒有金文
痿沒有金文
嚅沒有金文
谿沒有金文
繒沒有金文
氘沒有金文
阨沒有金文
俓沒有金文
娸沒有金文
痐沒有金文
紺沒有金文
焱沒有金文
猢沒有金文
廋沒有金文
椳沒有金文
赨沒有金文
靸沒有金文
僯沒有金文
熞沒有金文
瘜沒有金文
蝮沒有金文
歕沒有金文
鵂沒有金文
嚝沒有金文
譠沒有金文
鷈沒有金文
驒沒有金文
鷿沒有金文
祆沒有金

## 查漏補缺

In [24]:
# GB2312全部單字
GB2312 = []
with open('GB2312.txt','r',encoding='utf8') as gb:
    lines = gb.readlines()
    for line in lines:
        for word in line.strip():
            GB2312.append(word)

In [25]:
converter = opencc.OpenCC('t2s.json')
# converter.convert('汉字')  # 漢字
zipin_s = []
for char in 字頻:
    char_s = converter.convert(char)
    zipin_s.append(char_s)
zipin_lack = list(set(GB2312).difference(set(zipin_s)))
# 字頻中所有字符簡化，求差，備用（需要補充爬取，雖然幾乎不會有字）

In [26]:
print(len(zipin_lack))
print(zipin_lack)

1549
['镤', '窀', '墒', '篥', '蒗', '钕', '缣', '裣', '饧', '宀', '葜', '喾', '楦', '螋', '瀹', '劬', '逡', '橐', '岣', '爝', '冫', '馑', '茭', '跬', '伲', '鹑', '荭', '鲼', '醅', '谘', '鹋', '荑', '儋', '钯', '螗', '碛', '亠', '搠', '貊', '诖', '瘿', '瘃', '蜞', '禚', '讠', '蠖', '瓿', '眍', '郄', '绂', '桊', '莩', '圩', '祧', '莨', '谰', '僖', '冁', '剀', '歆', '跗', '鲇', '砩', '纩', '狻', '镩', '荜', '瘕', '聍', '珙', '毪', '舭', '酞', '邗', '倮', '纟', '脒', '嘞', '猃', '蝤', '粢', '睃', '瘛', '毂', '埚', '簏', '仨', '媪', '鞔', '躜', '躔', '舾', '礓', '岽', '潍', '蕖', '粜', '峄', '铞', '镥', '汆', '桁', '芏', '嗬', '痖', '锔', '酃', '桉', '榘', '蹯', '镨', '聃', '钇', '漭', '龋', '攮', '蝥', '嗾', '妪', '罱', '疖', '煺', '茏', '觏', '腈', '蛲', '鳇', '茛', '铯', '淝', '渖', '溆', '襻', '蘼', '栳', '猓', '萘', '晗', '蔹', '檑', '悃', '娈', '旆', '俜', '挢', '乾', '滹', '嵬', '攴', '厍', '酽', '祛', '艿', '黢', '谫', '儇', '眙', '镦', '枨', '钬', '仫', '陬', '徭', '煸', '钎', '豢', '鲎', '脬', '郐', '苈', '犰', '趿', '啶', '鳙', '溴', '洇', '毖', '彡', '嗥', '磔', '箢', '涑', '耱', '恝', '郧', '咣', '肀', '徂', '耋', '琊', '噍', '馕', '蝰', '滠', '缒', '瞀', '赇', '溘', '幞',

In [29]:
s2t_converter = opencc.OpenCC('s2t.json')

for zi_s in zipin_lack:
    zi = s2t_converter.convert(zi_s)
    # 先转成繁体字，按照繁体字比较靠谱些
    # 反正这些是之前字频里面没有的
    # 早知道就直接用GB2312爬了
    # 最初只是想搞几个常用字而已
    # 没想到后来心变大了
    zi_parse = urllib.parse.quote(zi)
    url = 'https://www.zdic.net/zd/zx/jw/' + zi_parse
    r = requests.get(url,headers=headers)
    text = r.text
    soup = BeautifulSoup(text, 'html.parser')
    try:
        jw = soup.find(name='div',attrs={"class": "zy1"})
        img_list = jw.find_all(name='img')
        
        headers['Referer'] = url
        # 更新请求头，否则403
        
        for img in img_list[0:1]:
            # 暫時只要第一個
            # 其实以后可以做一个alternative字体
            # 很多英文字体都有防止重复出现的替换字型
            
            src = 'https:'+ img.get('data-original')
            res = requests.get(url=src,headers=headers)
            filename = zi+'.svg'
            with open(filename,'wb') as f:
                f.write(res.content)        
    except:
        print(str(zipin_lack.index(zi_s)) + zi + '沒有金文') # 手動查索引太累了
        time.sleep(15)
        continue
    time.sleep(8)

0鏷沒有金文
1窀沒有金文


KeyboardInterrupt: 

# 鏡像給簡體字

In [30]:
一簡對一繁之簡體字 = []
一簡對多繁之簡體字 = []
with open('STCharacters.txt','r',encoding='utf8') as st:
    lines = st.readlines()
    for line in lines:
        a_dict = line.split('\t')
        k_s = a_dict[0]
        v_t = a_dict[1].split(' ')
        if len(v_t) == 1:
            一簡對一繁之簡體字.append(k_s)
        else:
            一簡對多繁之簡體字.append(k_s)

In [31]:
s2t_dict = {}
with open('STCharacters.txt','r',encoding='utf8') as st:
    lines = st.readlines()
    for line in lines:
        a_dict = line.split('\t')
        s = a_dict[0]
        t = a_dict[1].strip()
        s2t_dict[s] = t

In [39]:
L = []
for home, dirs, files in os.walk('去水印'): 
     for file in files:
        L.append(os.path.join(home, file))
# len(L)
list_of_svg_char = []
for l in L:
    char = l.split('.')[0][-1]
    list_of_svg_char.append(char)
# a = 0
will_be_mirrored_s_char = []
for s in list_of_svg_char:
    ss = converter.convert(s)
    if ss not in 一簡對多繁之簡體字 and ss not in list_of_svg_char:
#     if s2t_dict[s] in list_of_svg_char:
#         a += 1
#         if s not in list_of_svg_char:
        # 要保證這個簡體字型不僅有對應繁體字，
        # 還要保證它本身不在已有的svg之内
        will_be_mirrored_s_char.append(ss)
            # 然後才鏡像之
# print(a)
len(will_be_mirrored_s_char)
# 之後在js裏面鏡像，不複製本地文件
# 在最後輸出之前要先去水印的
# # 具體：
# for s_char in will_be_mirrored_s_char:
#     jscode = "font.setSvg('" + s_char + "', fs.readFileSync('去水印/" + s2t_dict[s] + ".svg').toString())\n"

522

# 去掉水印

In [33]:
# 將爬取的全部文件放到“去水印”文件夾中
# https://blog.csdn.net/qq_37174526/article/details/89489212
from xml.dom.minidom import parse

def 去水印(filename):
    svg = parse(filename)
    rootNode = svg.documentElement
#     print(rootNode.nodeName)
    gs = rootNode.getElementsByTagName("g")
    for g in gs:
        if g.hasAttribute('fill'):
            color = g.getAttribute('fill')
            if color[-2:] in ['fa','fb','fc']:
#                 print(color)
                rootNode.removeChild(g)

    path = rootNode.getElementsByTagName("path")
    for p in path:
        if p.hasAttribute('fill'):
            color = p.getAttribute('fill')
    #         print(color[-2:])
            if color[-2:] in ['fa','fb','fc']:
#                 print(color)
                rootNode.removeChild(p)
    # print(rootNode.toxml())
    with open(filename,'w') as f:
        f.write(rootNode.toxml())
        
## 注釋：
# 水印的位置和中英文是隨機的，但顔色是近似的，故而利用fill的顔色來判斷是否是水印
# 在瀏覽器中不可見，但是在轉換爲字體之後，會顯現出顔色（單色）
# 所以利用xml的優勢去除水印

In [34]:
# 獲取所有文件名
# 懶得寫代碼，沒有確認文件擴展名，
# 所以需要手動刪除測試中生成的其他文件
# L = []
# for home, dirs, files in os.walk('去水印'): 
#      for file in files:
#         L.append(os.path.join(home, file))
# 這一步已經挪到了上面


len(L)
# https://www.cnblogs.com/frisk/p/11567175.html

1967

In [35]:
for l in L:
    去水印(l)

# 輸出Node.js代碼

In [44]:
with open('create_font.js','w',encoding='utf8') as f:
    f.write('''
var fontCarrier = require('font-carrier')\n
var font = fontCarrier.create()\n
var fs = require('fs')\n
''')
    # https://stackoverflow.com/questions/29835440/meteor-referenceerror-fs-is-not-defined
    for l in L:
        char = l.split('.')[0][-1]
        jscode = "font.setSvg('" + char + "', fs.readFileSync('" + l.replace('\\','/') + "').toString())\n"
        f.write(jscode)
    for s_char in will_be_mirrored_s_char:
        t_char = s2t_converter.convert(s_char)
        if t_char in list_of_svg_char:
            jscode = "font.setSvg('" + s_char + "', fs.readFileSync('去水印/" + t_char + ".svg').toString())\n"
            f.write(jscode)
# 先注釋掉，輸出A式，搞完A式之後輸出B式
    f.write('''
font.output({
    path: './iconfont'
})
    ''')

# 後續操作
打開PowerShell, 安裝font-carrier后
輸入
```
node create_font.js
```
即可獲得輸出的多種格式的字體文件